In [1]:
import pandas as pd
from pathlib import Path
from math import floor
from torch import Tensor
from numpy import newaxis
from sklearn.model_selection import train_test_split
from assistive_funcs import filtering_image, check_ssim, check_gmsd, convert_to_grayscale, get_dataset_name
from torch.utils.data import DataLoader, Dataset
# Paths
main_data_path = Path("../data")
scv_folder = main_data_path / "csv_files"  # scv_folder
img_path = main_data_path / "images"


d:\Projects\PythonProjects\_venvs\3.9_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Paths
p_main_data = Path("../data")
p_models = Path("../models")

p_scv_folder = p_main_data / "csv_files" # datasets_path
p_img = p_main_data / "images"

p_noised_imgs = p_main_data / "FC_imgs_with_noise"

p_filtered_images = p_main_data / "FC_filtered_images"

p_gray_images = p_main_data / "gray_images"

In [3]:
win_size = 3
step = 20

dataset_name = get_dataset_name(win_size, step, p_scv_folder) #r"W5_S1_L3696640.csv"
dataset_name

'W3_S20_L9000.csv'

In [4]:
class _CustomSmallDataLoader(Dataset):
    def __init__(self, x, y) -> None:
        super().__init__()
        self.x = Tensor(x).float()
        self.y = Tensor(y[:, newaxis]).float()
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)
        

In [5]:
def get_train_test_small_data(*, scv_folder, dataset_name, batch_size, train_size):
    main_path = scv_folder / dataset_name
    data = pd.read_csv(main_path, header=None)
    y = data[data.shape[1] - 1].to_numpy()
    x = data.iloc[:, 0:data.shape[1]-1].to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=1-train_size, shuffle=False)
    train_data = DataLoader(_CustomSmallDataLoader(X_train, y_train), batch_size=batch_size)
    test_data = DataLoader(_CustomSmallDataLoader(X_test, y_test), batch_size=batch_size)
    return train_data, test_data
    

In [6]:
train_data, test_data = get_train_test_small_data(scv_folder=p_scv_folder, dataset_name=dataset_name, batch_size=128, train_size=0.7)

In [7]:
for x, y in train_data:
    print(x.shape, y.shape)
    break

torch.Size([128, 9]) torch.Size([128, 1])
